In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
import shutil

In [8]:
def get_roi(xy, img):
    #适用于长宽相等的图片
    size = img.shape[0]
    if len(xy[0]) > 1:
        min_x = np.min(xy[0])
        max_x = np.max(xy[0]) + 1
        max_x = max_x if max_x <= size else size
    else:
        min_x = 0 
        max_x = size
    if len(xy[1]) > 1:
        min_y = np.min(xy[1])
        max_y = np.max(xy[1]) + 1
        max_y = max_y if max_y <= size else size
    else:
        min_y = 0
        max_y = size
    roi_img = img[min_x:max_x, min_y:max_y]
    return roi_img

In [9]:
def extract_roi_from_gray(img_dir, save_dir):
    if os.path.exists(save_dir):
        shutil.rmtree(save_dir)
        os.makedirs(save_dir)
    else:
        os.makedirs(save_dir)
    for filename in os.listdir(img_dir):
#         print(filename)
        img = cv2.imread(f'{img_dir}/{filename}')
        size = img.shape[0]
        img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        bit = img_gray <= 150
        mask = img_gray * bit
        kernel = np.ones((3,3), np.uint8) 
        img_erosion = cv2.erode(mask, kernel, iterations=20) 
        img_erosion_dia = cv2.dilate(img_erosion, kernel,iterations=20)
        xy = np.nonzero(img_erosion_dia)
        roi_img = get_roi(xy, img)
#         print(img.shape, "roi_img's size", roi_img.shape)
        roi_img = cv2.resize(roi_img, (size, size))
        cv2.imwrite(f'{save_dir}/{filename}', roi_img)

In [11]:
# result = cv2.bitwise_and(g, g, mask=np.array(k))
# plt.imshow(result)
def extract_roi(cam_path, img_dir):
    cam = cv2.imread(cam_path)
    if cam_path[-24] != 'I':
        img_path = img_dir + "/" + cam_path[-36:-8]
    else:
        img_path = img_dir + "/" + cam_path[-24:-8]
#     print(img_path, "************")
#     print(img_path, "KKKKKKK")
    img = cv2.imread(img_path)
    size = img.shape[0]
#     print(img.shape,"iiiiii")
    gray_cam = cv2.cvtColor(cam, cv2.COLOR_BGR2GRAY)
    #140 is value of gray, random select according of color
    bit = gray_cam > 140
    mask = gray_cam * bit
#     print(mask)
    xy = np.nonzero(mask)
    roi_img = get_roi(xy, img)
    print(cam.shape, "roi_img's size", roi_img.shape)
    roi_img = cv2.resize(roi_img, (size, size))
    return roi_img

In [12]:
def cam_to_img(cam_dir, img_dir, save_dir, have_tran_dir):
    #用这个方法的前提是cam是要足够准确
    #have_tran_dir可要可不要，只是为了看看已经转换好的图片
#     num = 0
    if os.path.exists(save_dir):
        shutil.rmtree(save_dir)
        os.makedirs(save_dir)
    else:
        os.makedirs(save_dir)
    for filename in os.listdir(cam_dir):
        cam_path = cam_dir + "/" + filename
        img = extract_roi(cam_path, img_dir)
        if filename[12] != '.':
            cv2.imwrite(f'{save_dir}/{filename[:28]}', img)
        else:
            cv2.imwrite(f'{save_dir}/{filename[:16]}', img)

In [ ]:
# cam_dir = '/home/xhzhu/mycode/siim_code/all_img_cam'
img_dir = '/home/xhzhu/data/my_siim/512x512-dataset-melanoma/512x512-dataset-melanoma'
save_dir = './all_roi_img'
have_tran_dir = './have_converted'
#cam_to_img(cam_dir=cam_dir, img_dir=img_dir, save_dir=save_dir, have_tran_dir=have_tran_dir)
extract_roi_from_gray(img_dir, save_dir)